In [7]:
# !pip install torch==2.5.0
# !pip install numpy==1.26.4
# !pip install openai==1.79.0
# !pip install tenacity==9.1.2
# !pip install tiktoken==0.9.0
# !pip install transformers==4.51.3
# !pip install pandas==2.2.3
# !pip install scikit-learn==1.6.1
# !pip install bitsandbytes==0.45.5
# !pip install datasets==3.6.0
# !pip install sentencepiece==0.2.0
# !pip install peft==0.15.2
# !pip install evaluate==0.4.3
# !pip install trl==0.11.4
# !pip install protobuf==6.31.0
# !pip install python-dotenv==1.1.0
# !pip install pandas_ta
# !pip install ollama==0.4.8
# !pip install accelerate==1.7.0
# !pip install ipywidgets
# !pip install pynvml==8.1.7
# !pip uninstall torch torchvision torchaudio -y
# !pip install torch==2.5.0 torchvision==0.20.0 torchaudio==2.5.0 --index-url https://download.pytorch.org/whl/cu124

In [3]:
import pkg_resources

libs = [
    "numpy", "openai", "tenacity", "tiktoken", "transformers", "pandas",
    "scikit-learn", "torch", "bitsandbytes", "datasets", "sentencepiece",
    "peft", "evaluate", "trl", "protobuf", "python-dotenv", "pandas_ta",
    "ollama", "accelerate", "ipywidgets"
]

for lib in libs:
    try:
        version = pkg_resources.get_distribution(lib).version
        print(f"{lib}=={version}")
    except pkg_resources.DistributionNotFound:
        print(f"{lib} not installed")

numpy==1.26.4
openai==1.79.0
tenacity==9.1.2
tiktoken==0.9.0
transformers==4.51.3
pandas==2.2.3
scikit-learn==1.6.1
torch==2.5.0+cu124
bitsandbytes==0.45.5
datasets==3.6.0
sentencepiece==0.2.0
peft==0.15.2
evaluate==0.4.3
trl==0.11.4
protobuf==6.31.0
python-dotenv==1.1.0
pandas_ta==0.3.14b0
ollama==0.4.8
accelerate==1.7.0
ipywidgets==8.1.7


/tmp/ipykernel_5990/316775683.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [4]:
import torch # type: ignore
print(torch.cuda.is_available())  # Nếu trả về False, CUDA chưa hoạt động
print(torch.cuda.device_count())  # Kiểm tra số lượng GPU
print(torch.cuda.get_device_name(0))  # Hiển thị tên GPU
# print(torch.set_default_device())

True
1
GRID P40-24Q


In [1]:
import argparse
import torch
import numpy as np
import random
from peft import (
    LoraConfig,
    get_peft_model
)
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import os
import sys
import json
import transformers
import warnings
from datasets import load_dataset
from predict_module import sft_dataloader

# Thiết lập seed
fix_seed = 100
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

# Cấu hình tham số cho huấn luyện
args = argparse.Namespace(
    price_dir="data/sample_price/preprocessed/",  # Thư mục dữ liệu giá
    tweet_dir="data/sample_tweet/raw/",  # Thư mục dữ liệu tweet
    seq_len=5,  # Độ dài chuỗi đầu vào
    technical_indicator_dir="data/sample_price/technical_indicator/",
    llm_summarize="OpenAILLM", # OpenAILLM // DeepSeekLLM
    wandb=False,  # Tắt logging với Weights & Biases
    data_path="./data/merge_sample.json",  # Đường dẫn file dữ liệu
    output_path="./saved_models/lora-DeepSeek-R1-Distill-Qwen",  # Thư mục lưu mô hình LoRA
    model_path="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",  # Mô hình DeepSeek
    eval_steps=200,  # Số bước đánh giá
    save_steps=200,  # Số bước lưu checkpoint
    resume_from_supervised_checkpoint=None,  # Không resume từ checkpoint
    ignore_data_skip="False",  # Không bỏ qua dữ liệu khi resume
    num_reflect_trials=2,  # Số lần thử phản ánh
    datasets_dir="./datasets/",  # Thư mục datasets
    local_rank=0,  # Rank cục bộ cho DDP
    resume_from_reward_checkpoint=False,  # Không resume từ reward checkpoint
    deepspeed=None,  # Không dùng DeepSpeed
    per_device_train_batch_size=4,  # Batch size huấn luyện trên mỗi GPU
    per_device_eval_batch_size=4,  # Batch size đánh giá trên mỗi GPU
    reward_gradient_accumulation_steps=8,  # Số bước tích lũy gradient cho reward
    reward_learning_rate=3e-5,  # Learning rate cho reward
    weight_decay=0.001,  # Trọng số giảm dần
    reward_base_model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",  # Mô hình reward
    bf16=False,  # Sử dụng fp16 thay vì bf16
    num_train_epochs=2,  # Số epoch huấn luyện
    train_subset=100000,  # Số mẫu huấn luyện
    eval_subset=50000,  # Số mẫu đánh giá
    gradient_checkpointing=True,  # Bật gradient checkpointing để tiết kiệm VRAM
    optim="adamw_torch",  # Optimizer AdamW từ PyTorch
    lr_scheduler_type="cosine",  # Lịch trình learning rate kiểu cosine
    reward_adapter="./saved_models/reward_model_deepseek-r1-distill-qwen",  # Adapter reward
    rl_base_model="./saved_models/lora-DeepSeek-R1-Distill-Qwen-adapter-merged",  # Mô hình RL
    tokenizer_name="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",  # Tokenizer
    reward_model_name="./saved_models/reward_model_deepseek-r1-distill-qwen-adapter-merged",  # Mô hình reward merged
    log_with=None,  # Không dùng logging cụ thể
    rl_learning_rate=2e-5,  # Learning rate cho RL
    output_max_length=128,  # Độ dài đầu ra tối đa
    mini_batch_size=4,  # Kích thước mini-batch
    batch_size=128,  # Kích thước batch tổng
    ppo_epochs=4,  # Số epoch cho PPO
    rl_gradient_accumulation_steps=32,  # Số bước tích lũy gradient cho RL
    adafactor=False,  # Không dùng Adafactor
    early_stopping=True,  # Bật early stopping
    target_kl=0.1,  # KL target cho RL
    reward_baseline=0,  # Baseline cho reward
    batched_gen=True,  # Tạo batch
    save_freq=None,  # Tần suất lưu
    output_dir="./saved_models/tuning_deepseek_r1_distill_qwen_checkpoints/",  # Thư mục lưu checkpoint
    seed=0,  # Seed cho RL
    num_shots=4,  # Số shots cho few-shot
    save_dir="results/"  # Thư mục lưu kết quả
)


print("Args in experiment:")
print(args)

args.data_path


Args in experiment:
Namespace(price_dir='data/sample_price/preprocessed/', tweet_dir='data/sample_tweet/raw/', seq_len=5, technical_indicator_dir='data/sample_price/technical_indicator/', llm_summarize='OpenAILLM', wandb=False, data_path='./data/merge_sample.json', output_path='./saved_models/lora-DeepSeek-R1-Distill-Qwen', model_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', eval_steps=200, save_steps=200, resume_from_supervised_checkpoint=None, ignore_data_skip='False', num_reflect_trials=2, datasets_dir='./datasets/', local_rank=0, resume_from_reward_checkpoint=False, deepspeed=None, per_device_train_batch_size=4, per_device_eval_batch_size=4, reward_gradient_accumulation_steps=8, reward_learning_rate=3e-05, weight_decay=0.001, reward_base_model='deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', bf16=False, num_train_epochs=2, train_subset=100000, eval_subset=50000, gradient_checkpointing=True, optim='adamw_torch', lr_scheduler_type='cosine', reward_adapter='./saved_models/reward_model

'./data/merge_sample.json'

In [ ]:
# import torch
# import torch.nn as nn
# from datasets import load_dataset

# DATA_PATH = args.data_path

# data = load_dataset("json", data_files=DATA_PATH)
# data['train'][1]

# # --- Tải tokenizer ---
# tokenizer = AutoTokenizer.from_pretrained(
#     args.model_path,
#     add_eos_token=True,  # Thêm token kết thúc
#     local_files_only=args.offline if hasattr(args, 'offline') else False,  # Chế độ offline
#     trust_remote_code=True  # Cho DeepSeek
# )
# if tokenizer.pad_token is None:
#     tokenizer.pad_token = tokenizer.eos_token  # Gán pad_token


# CUTOFF_LEN = 256  # Độ dài chuỗi tối đa
# data = load_dataset("json", data_files=DATA_PATH)  # Tải JSON dataset
# val_set_size = int(0.1 * len(data["train"]))  # Tính validation size
# print(data)  # In thông tin dataset

# # --- Tải dữ liệu huấn luyện và đánh giá ---
# dataloader = sft_dataloader.SFTDataLoader(data, CUTOFF_LEN, val_set_size, tokenizer)  # Định dạng và tokenize
# train_data, val_data = dataloader.load_data()  # Chia train/validation
# train_data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 33
    })
})


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output', 'input_ids', 'labels', 'attention_mask'],
    num_rows: 30
})

In [2]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
)
# from trl import SFTTrainer
import torch
from peft import LoraConfig, get_peft_model, set_peft_model_state_dict


def supervised_finetune(args):
    # --- Các hằng số huấn luyện ---
    MICRO_BATCH_SIZE = args.per_device_train_batch_size  # Batch size mỗi GPU
    BATCH_SIZE = args.batch_size  # Batch size tổng
    MAX_STEPS = None  # Số bước tối đa, tính động
    GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE  # Bước tích lũy gradient
    EPOCHS = args.num_train_epochs  # Số epoch
    LEARNING_RATE = 3e-4  # Tốc độ học
    CUTOFF_LEN = 256  # Độ dài chuỗi tối đa
    LORA_R = 16  # Rank LoRA
    LORA_ALPHA = 32  # Hệ số scale LoRA
    LORA_DROPOUT = 0.05  # Dropout LoRA
    VAL_PCT = 0.1  # Tỷ lệ validation
    TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj"]  # Layer áp dụng LoRA
    DATA_PATH = args.data_path  # Đường dẫn dữ liệu
    OUTPUT_DIR = args.output_path  # Thư mục lưu mô hình
    world_size = int(os.environ.get("WORLD_SIZE", 1))  # Số GPU (DDP)


    # --- Xử lý DDP ---
    ddp = world_size != 1  # Kiểm tra đa GPU
    if ddp:
        torch.cuda.set_device(int(os.environ.get("LOCAL_RANK", 0)))  # Gán GPU
        GRADIENT_ACCUMULATION_STEPS = GRADIENT_ACCUMULATION_STEPS // world_size  # Chia tích lũy gradient

    #==============================================================================================================================
    print(f"Đang tải mô hình từ: {args.model_path}")  # In đường dẫn mô hình

    # Step 3: Load model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(
        args.model_path,
        add_eos_token=True,  # Thêm token kết thúc
        local_files_only=args.offline if hasattr(args, 'offline') else False  # Chế độ offline
        )

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token  # Gán pad_token

    model = AutoModelForCausalLM.from_pretrained(
        args.model_path, 
        torch_dtype=torch.float16, 
        device_map="auto")

    # --- Tải dữ liệu ---
    dataset = load_dataset("json", data_files=DATA_PATH)  # Tải JSON dataset


    val_set_size = int(VAL_PCT * len(dataset["train"]))  # Tính validation size
    print(dataset)  # In thông tin dataset


    # --- Tải dữ liệu huấn luyện và đánh giá ---
    dataloader = sft_dataloader.SFTDataLoader(dataset, CUTOFF_LEN, val_set_size, tokenizer)  # Định dạng và tokenize
    train_data, val_data = dataloader.load_data()  # Chia train/validation
    train_data, val_data

    # Step 4: Configure LoRA
    peft_config = LoraConfig(
        r=LORA_R,  # Rank LoRA
        lora_alpha=LORA_ALPHA,  # Scale LoRA
        target_modules=TARGET_MODULES,  # Layer LoRA
        lora_dropout=LORA_DROPOUT,  # Dropout
        bias="none",  # Không bias
        task_type="CAUSAL_LM"  # Tác vụ ngôn ngữ
    )


    model = get_peft_model(model, peft_config)


    # --- Tính max_steps ---
    now_max_steps = max((len(dataset["train"]) - val_set_size) // BATCH_SIZE * EPOCHS, EPOCHS)  # Số bước tối đa


    # --- Xử lý checkpoint ---
    if args.resume_from_supervised_checkpoint:  # Nếu có checkpoint
        checkpoint_name = os.path.join(args.resume_from_supervised_checkpoint, "pytorch_model.bin")  # Đường dẫn checkpoint
        if not os.path.exists(checkpoint_name):
            pytorch_bin_path = checkpoint_name
            checkpoint_name = os.path.join(args.resume_from_supervised_checkpoint, "adapter_model.bin")  # Kiểm tra file khác
            if os.path.exists(checkpoint_name):
                os.rename(checkpoint_name, pytorch_bin_path)  # Đổi tên
                warnings.warn("Đã đổi tên 'adapter_model.bin' thành 'pytorch_model.bin'")
            else:
                args.resume_from_supervised_checkpoint = None  # Bỏ resume
        if os.path.exists(checkpoint_name):
            print(f"Tiếp tục từ: {checkpoint_name}")
            adapters_weights = torch.load(checkpoint_name)  # Tải LoRA
            model = set_peft_model_state_dict(model, adapters_weights)  # Áp dụng
        else:
            print(f"Không tìm thấy: {checkpoint_name}")
        train_args_path = os.path.join(args.resume_from_supervised_checkpoint, "trainer_state.json")  # File trạng thái
        if os.path.exists(train_args_path):
            base_train_args = json.load(open(train_args_path, 'r'))
            base_max_steps = base_train_args["max_steps"]  # Số bước cũ
            resume_scale = base_max_steps / now_max_steps
            if base_max_steps > now_max_steps:
                warnings.warn(f"Thay epoch {EPOCHS} bằng {base_max_steps}")
                EPOCHS = None
                MAX_STEPS = base_max_steps
            else:
                MAX_STEPS = now_max_steps
    else:
        MAX_STEPS = now_max_steps


    # Step 5: Define training arguments
    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,  # Directory to save results
        per_device_train_batch_size=MICRO_BATCH_SIZE,  # Batch size GPU
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,  # Tích lũy gradient
        warmup_steps=100,  # Bước khởi động
        num_train_epochs=EPOCHS if EPOCHS else 1,  # Epoch
        max_steps=MAX_STEPS,  # Số bước tối đa
        learning_rate=LEARNING_RATE,  # Tốc độ học
        bf16=args.bf16,  # BF16
        fp16=not args.bf16,  # FP16
        logging_steps=20,  # Log mỗi 20 bước
        eval_strategy="steps" if val_set_size > 0 else "no",  # Đánh giá
        save_strategy="steps",  # Lưu checkpoint
        eval_steps=args.eval_steps if val_set_size > 0 else None,  # Bước đánh giá
        save_steps=args.save_steps,  # Bước lưu
        save_total_limit=30,  # Số checkpoint tối đa
        load_best_model_at_end=True if val_set_size > 0 else False,  # Tải mô hình tốt
        ddp_find_unused_parameters=False if ddp else None,  # Tối ưu DDP
        report_to="wandb" if args.wandb else [],  # Báo cáo WandB
        optim=args.optim,  # Bộ tối ưu
        lr_scheduler_type=args.lr_scheduler_type,  # Scheduler
        remove_unused_columns=True,  # Xóa cột thừa
        max_grad_norm=1.0,  # Giới hạn gradient
    )


    # Step 6: Initialize the trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_data, 
        eval_dataset=val_data,  # Small evaluation set
        data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    )

    # Step 7: Train the model
    trainer.train(resume_from_checkpoint=args.resume_from_supervised_checkpoint)
    
    model.save_pretrained(OUTPUT_DIR) 

# --- Chạy huấn luyện ---

supervised_finetune(args)

from predict_module.merge_peft_adapter import merge_peft_adapter

# --- Gộp adapter LoRA ---
merge_peft_adapter(model_name=args.output_path, output_name=args.rl_base_model)

Đang tải mô hình từ: deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B


tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

...

Map:   0%|          | 0/63 [00:00<?, ? examples/s]

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss


Cấu hình LoRA: LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', revision=None, inference_mode=True, r=16, target_modules={'v_proj', 'k_proj', 'q_proj', 'o_proj'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)
Lưu mô hình gộp tại: ./saved_models/lora-DeepSeek-R1-Distill-Qwen-adapter-merged


In [5]:
import torch

# Giải phóng bộ nhớ trên GPU
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [2]:
import torch
import torch.nn as nn
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    HfArgumentParser,
    PreTrainedTokenizerBase,
    Trainer,
    TrainingArguments,
)
from peft import LoraConfig, TaskType, get_peft_model
import evaluate
import numpy as np
from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Union
from transformers.utils import PaddingStrategy

from predict_module import rm_dataloader

def train_reward_model(args):
    script_args = args
    dataset_name = script_args.datasets_dir
    print("dataset_name:", dataset_name)
    
    output_name = script_args.reward_adapter
    
    training_args = TrainingArguments(
        output_dir=output_name,
        learning_rate=script_args.reward_learning_rate,
        per_device_train_batch_size=1,  # Giảm batch size
        per_device_eval_batch_size=1,
        num_train_epochs=script_args.num_train_epochs,
        weight_decay=script_args.weight_decay,
        eval_strategy="steps",
        eval_steps=200,
        save_strategy="steps",
        save_steps=200,
        save_total_limit=2,
        gradient_accumulation_steps=16,  # Tăng gradient accumulation
        gradient_checkpointing=False,  # Tắt gradient checkpointing
        deepspeed=None,  # Tắt DeepSpeed để kiểm tra
        remove_unused_columns=False,
        label_names=[],
        logging_strategy="steps",
        logging_steps=10,
        optim=script_args.optim,
        lr_scheduler_type=script_args.lr_scheduler_type,
        report_to="none",
        no_cuda=False,  # Đảm bảo dùng GPU
        bf16=True,
    )
    
    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(script_args.reward_base_model, trust_remote_code=True)
    model = AutoModelForSequenceClassification.from_pretrained(
        script_args.reward_base_model,
        num_labels=1,
        torch_dtype=torch.bfloat16,  # Thử bfloat16
        trust_remote_code=True,
    )
    
    # # Handle pad token
    # if tokenizer.pad_token is None:
    #     tokenizer.pad_token = tokenizer.eos_token
    #     model.config.pad_token_id = tokenizer.eos_token_id
    # else:
    #     model.config.pad_token_id = tokenizer.pad_token_id
    
    # Set device
    # device_map = "cuda:0" if torch.cuda.is_available() else "cpu"
    # print("device_map:", device_map)
    # model = model.to(device_map)  # Chuyển thủ công
    
    # LoRA config
    peft_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        inference_mode=False,
        r=4,  # Giảm r
        lora_alpha=8,
        lora_dropout=0.05,
        bias="none",
    )
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    
    num_proc = 1
    reward_dataloder = rm_dataloader.RewardDataLoader(dataset_name, script_args.train_subset, script_args.eval_subset, num_proc, tokenizer)
    train_dataset, eval_dataset = reward_dataloder.load_data()
    
    @dataclass
    class RewardDataCollatorWithPadding:
        tokenizer: PreTrainedTokenizerBase
        padding: Union[bool, str, PaddingStrategy] = True
        max_length: Optional[int] = None
        pad_to_multiple_of: Optional[int] = None
        return_tensors: str = "pt"

        def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Any]:
            features_j = []
            features_k = []
            for feature in features:
                features_j.append(
                    {
                        "input_ids": feature["input_ids_j"],
                        "attention_mask": feature["attention_mask_j"],
                    }
                )
                features_k.append(
                    {
                        "input_ids": feature["input_ids_k"],
                        "attention_mask": feature["attention_mask_k"],
                    }
                )
            batch_j = self.tokenizer.pad(
                features_j,
                padding=self.padding,
                max_length=self.max_length,
                pad_to_multiple_of=self.pad_to_multiple_of,
                return_tensors=self.return_tensors,
            )
            batch_k = self.tokenizer.pad(
                features_k,
                padding=self.padding,
                max_length=self.max_length,
                pad_to_multiple_of=self.pad_to_multiple_of,
                return_tensors=self.return_tensors,
            )
            batch = {
                "input_ids_j": batch_j["input_ids"],
                "attention_mask_j": batch_j["attention_mask"].to(dtype=torch.bfloat16),
                "input_ids_k": batch_k["input_ids"],
                "attention_mask_k": batch_k["attention_mask"].to(dtype=torch.bfloat16),
                "return_loss": True,
            }
            return batch
    
    accuracy = evaluate.load("accuracy")
    
    def compute_metrics(eval_pred):
        predictions, _ = eval_pred
        predictions = np.argmax(predictions, axis=0)
        labels = np.zeros(predictions.shape)
        return accuracy.compute(predictions=predictions, references=labels)
    
    class RewardTrainer(Trainer):
        def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
            rewards_j = model(
                input_ids=inputs["input_ids_j"], attention_mask=inputs["attention_mask_j"])[0]
            rewards_k = model(
                input_ids=inputs["input_ids_k"], attention_mask=inputs["attention_mask_k"])[0]
            loss = -nn.functional.logsigmoid(rewards_j - rewards_k).mean()
            if return_outputs:
                return loss, {"rewards_j": rewards_j, "rewards_k": rewards_k}
            return loss
    
    trainer = RewardTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
        data_collator=RewardDataCollatorWithPadding(
            tokenizer=tokenizer, max_length=512, pad_to_multiple_of=8),
    )
    
    model.config.use_cache = True
    trainer.train(script_args.resume_from_reward_checkpoint)
    
    print("Saving last checkpoint of the model")
    model.save_pretrained(output_name)

train_reward_model(args)

from predict_module.merge_peft_adapter import merge_peft_adapter
merge_peft_adapter(model_name=args.reward_adapter, output_name=args.reward_model_name)

dataset_name: ./datasets/


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 546,304 || all params: 1,544,262,144 || trainable%: 0.0354
train_dataset:  3
train_dataset:  3
eval_dataset:  3
eval_dataset:  3


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/root/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2718: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss


Saving last checkpoint of the model
Cấu hình LoRA: LoraConfig(task_type='SEQ_CLS', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', revision=None, inference_mode=True, r=4, target_modules={'q_proj', 'v_proj'}, exclude_modules=None, lora_alpha=8, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=['classifier', 'score'], init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)
Lưu mô hình gộp tại: ./saved_models/reward_model_deepseek-r1-distill-qwen-adapter-merged


In [3]:
# Giải phóng bộ nhớ trên GPU
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [6]:
# from datasets import load_dataset
# from transformers import DataCollatorWithPadding, AutoTokenizer


# def build_dataset(tokenizer, dataset_name, input_min_text_length=2, input_max_text_length=8):
#     """
#     Tạo dataset cho huấn luyện PPO.
    
#     Args:
#         tokenizer: Tokenizer để mã hóa văn bản.
#         dataset_name: Tên hoặc đường dẫn dataset.
#         input_min_text_length: Độ dài tối thiểu của câu hỏi.
#         input_max_text_length: Độ dài tối đa của câu hỏi.
    
#     Returns:
#         Dataset đã được xử lý với các cột query và input_ids.
#     """
#     ds = load_dataset(dataset_name, split="train")
#     original_columns = ds.column_names
#     def preprocess_function(examples):
#         new_examples = {
#             "query": [],
#             "input_ids": [],
#         }
#         # Giả định dataset có cột 'user_input' hoặc 'question'
#         input_key = "user_input" if "user_input" in examples else "question"
#         for question in examples[input_key]:
#             query = "Question: " + question + "\n\nAnswer: "
#             tokenized_question = tokenizer(
#                 query,
#                 truncation=True,
#                 max_length=512,
#                 return_tensors="pt"
#             )
#             new_examples["query"].append(query)
#             new_examples["input_ids"].append(tokenized_question["input_ids"].squeeze(0))
#         return new_examples

#     ds = ds.map(
#         preprocess_function,
#         batched=True,
#         num_proc=1,
#         remove_columns=original_columns,
#     )
#     ds.set_format(type="torch")
#     return ds

#     # Tạo dataset
#     dataset = build_dataset(tokenizer, dataset_name=dataset_name)

# dataset_name = args.datasets_dir

# ds = load_dataset(dataset_name, split="train")
# ds

# tokenizer_name="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
# # Tải tokenizer
# tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trust_remote_code=True)
    
# dataset = build_dataset(tokenizer, dataset_name=dataset_name)

# dataset


In [4]:
from dataclasses import dataclass, field
from typing import Optional
import torch
from accelerate import Accelerator
from datasets import load_dataset, concatenate_datasets
from peft import LoraConfig
from tqdm import tqdm
from transformers import Adafactor, AutoTokenizer, AutoModelForSequenceClassification, AutoConfig, AutoModelForCausalLM, DataCollatorWithPadding
from transformers import GenerationConfig, pipeline
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer
from trl.core import LengthSampler
from trl import create_reference_model
import os

tqdm.pandas()

def tuning_lm_with_rl(args):
    # Khởi tạo script_args từ args
    script_args = args
    reward_model_name = script_args.reward_model_name
    print("reward_model_name:", reward_model_name)

    # Đường dẫn dataset
    dataset_name = script_args.datasets_dir
    print("dataset_name:", dataset_name)

    # Cấu hình PPO
    config = PPOConfig(
        learning_rate=script_args.rl_learning_rate,
        # batch_size=script_args.batch_size,
        batch_size=4,
        # mini_batch_size=script_args.mini_batch_size,
        mini_batch_size=2,
        # gradient_accumulation_steps=script_args.rl_gradient_accumulation_steps,
        gradient_accumulation_steps=1,
        ppo_epochs=script_args.ppo_epochs,  
        seed=script_args.seed,
    )

    # Tên mô hình gốc
    model_name = script_args.rl_base_model
    print("model_name:", model_name)

    # Tải dataset
    train_dataset = load_dataset(dataset_name, split="train")
    print("train_dataset size:", len(train_dataset))

    # Cấu hình tham số cho sentiment pipeline
    sent_kwargs = {
        "return_all_scores": True,
        "function_to_apply": "none",
        "batch_size": 1,
        "truncation": True
    }

    # Tải tokenizer
    tokenizer = AutoTokenizer.from_pretrained(script_args.tokenizer_name, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    print("Tokenizer loaded:", tokenizer.__class__.__name__)

    def build_dataset(tokenizer, dataset_name, input_min_text_length=2, input_max_text_length=8):
        """
        Tạo dataset cho huấn luyện PPO.
        
        Args:
            tokenizer: Tokenizer để mã hóa văn bản.
            dataset_name: Tên hoặc đường dẫn dataset.
            input_min_text_length: Độ dài tối thiểu của câu hỏi.
            input_max_text_length: Độ dài tối đa của câu hỏi.
        
        Returns:
            Dataset đã được xử lý với các cột query và input_ids.
        """
        ds = load_dataset(dataset_name, split="train")
        # Giả sử dataset của bạn tên là `ds`, hiện có 6 dòng
        repeat_factor = 64 // len(ds) + 1  # Lặp đủ số lần để vượt 64

        # Lặp lại nhiều lần rồi cắt còn đúng 64
        ds = concatenate_datasets([ds] * repeat_factor)
        ds = ds.select(range(64))  # Lấy đúng 64 dòng
        original_columns = ds.column_names

        def preprocess_function(examples):
            new_examples = {
                "query": [],
                "input_ids": [],
            }
            # Giả định dataset có cột 'user_input' hoặc 'question'
            input_key = "user_input" if "user_input" in examples else "question"
            for question in examples[input_key]:
                query = "Question: " + question + "\n\nAnswer: "
                tokenized_question = tokenizer(
                    query,
                    truncation=True,
                    max_length=512,
                    return_tensors="pt"
                )
                new_examples["query"].append(query)
                new_examples["input_ids"].append(tokenized_question["input_ids"].squeeze(0))
            return new_examples

        ds = ds.map(
            preprocess_function,
            batched=True,
            num_proc=1,
            remove_columns=original_columns,
        )
        ds.set_format(type="torch")
        return ds

    # Tạo dataset
    dataset = build_dataset(tokenizer, dataset_name=dataset_name)
    print("Dataset created with", len(dataset), "samples")

    def collator(data):
        """
        Collator để xử lý batch dữ liệu.
        """
        return dict((key, [d[key] for d in data]) for key in data[0])
    # def collator(data):
    #     """
    #     Custom collator cho PPOTrainer.
    #     Dữ liệu đầu vào là list[dict], mỗi dict chứa:
    #         - "query": str
    #         - "input_ids": Tensor (1D)
    #     """
    #     input_ids = [item["input_ids"] for item in data]
    #     queries = [item["query"] for item in data]

    #     # Pad input_ids thủ công nếu cần
    #     input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)

    #     return {
    #         "input_ids": input_ids,
    #         "query": queries,
    #     }

    # Đặt seed để đảm bảo tính tái lập
    torch.manual_seed(config.seed)

    # Cấu hình LoRA
    lora_config = LoraConfig(
        r=8,
        lora_alpha=16,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
    )

    # Tải mô hình chính với value head
    model = AutoModelForCausalLMWithValueHead.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,  # Đồng bộ với reward model
        peft_config=lora_config,
    )

    # Gán thủ công base_model_prefix để tránh lỗi
    model.base_model_prefix = "model"  # hoặc "transformer", tùy thuộc vào tên trong mô hình gốc
    # Gán generation_config để tránh lỗi AttributeError
    model.generation_config = GenerationConfig.from_pretrained(
        "./saved_models/lora-DeepSeek-R1-Distill-Qwen-adapter-merged"
    )
    # model.config.return_dict = True

    # try:
    #     model.base_model.model.config.return_dict = True
    # except:
    #     pass

    # model = get_peft_model(model, lora_config)


    # # Tải mô hình giá trị (value model) với value head
    # value_model = AutoModelForCausalLM.from_pretrained(
    #     model_name,
    #     torch_dtype=torch.bfloat16,
    #     trust_remote_code=True,
    # )
    # # value_model = get_peft_model(value_model, lora_config)
    # def force_return_dict_recursively(model):
    #     """
    #     Recursively patch forward function of all relevant submodules to enforce return_dict=True.
    #     """
    #     if hasattr(model, 'forward'):
    #         original_forward = model.forward

    #         def patched_forward(*args, **kwargs):
    #             kwargs["return_dict"] = True
    #             return original_forward(*args, **kwargs)

    #         model.forward = patched_forward

    #     # Patch model.base_model.model nếu có
    #     if hasattr(model, "base_model") and hasattr(model.base_model, "model"):
    #         force_return_dict_recursively(model.base_model.model)

    #     return model

    # # Sau khi load:
    # model = force_return_dict_recursively(model)
    # value_model = force_return_dict_recursively(value_model)




    # Tạo generation_config nếu không có
    if not hasattr(model, "generation_config"):
        model.generation_config = GenerationConfig.from_pretrained(model_name, trust_remote_code=True)

    print("Finetune model:", model_name, type(model))

    # # Tải reward model từ checkpoint
    # reward_model = AutoModelForSequenceClassification.from_pretrained(
    #     reward_model_name,
    #     torch_dtype=torch.bfloat16,
    #     trust_remote_code=True,
    # )
    # reward_model_config = AutoConfig.from_pretrained(reward_model_name, trust_remote_code=True)
    # print("Reward model:", type(reward_model))

    # Tạo optimizer (nếu dùng Adafactor)
    optimizer = None
    if script_args.adafactor:
        optimizer = Adafactor(
            filter(lambda p: p.requires_grad, model.parameters()),
            scale_parameter=False,
            relative_step=False,
            warmup_init=False,
            lr=config.learning_rate,
        )

    # Khởi tạo PPOTrainer
    print(dataset)
    ppo_trainer = PPOTrainer(
        config=config,  # Sử dụng args thay vì config
        model=model,
        tokenizer=tokenizer,  
        # reward_model=reward_model,
        # ref_model = None,
        # value_model=value_model,
        dataset=dataset,
        data_collator=collator,
        optimizer=optimizer,
    )
    
    # ppo_trainer.train()

    # Xác định thiết bị
    device = ppo_trainer.accelerator.device
    if ppo_trainer.accelerator.num_processes == 1:
        device = 0 if torch.cuda.is_available() else "cpu"
    print("Device:", device)

    # Tạo sentiment pipeline
    sentiment_pipe = pipeline(
        "sentiment-analysis",
        model=reward_model_name,
        device_map="auto",
        # config=reward_model_config,
        tokenizer=tokenizer,
        # device=device,
    )

    # Cấu hình tham số sinh văn bản
    generation_kwargs = {
        "top_k": 0.0,
        "top_p": 1.0,
        "do_sample": True,
        "pad_token_id": tokenizer.pad_token_id,
        "eos_token_id": tokenizer.eos_token_id,
    }
    output_min_length = 32
    output_max_length = script_args.output_max_length
    output_length_sampler = LengthSampler(output_min_length, output_max_length)

    # Vòng lặp huấn luyện PPO
    for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
        question_tensors = batch["input_ids"]

        # Sinh phản hồi
        response_tensors = ppo_trainer.generate(
            question_tensors,
            return_prompt=False,
            length_sampler=output_length_sampler,
            **generation_kwargs,
        )
        batch["response"] = tokenizer.batch_decode(response_tensors, skip_special_tokens=True)

        # Tính điểm thưởng từ reward model
        texts = [q + r for q, r in zip(batch["query"], batch["response"])]
        pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
        rewards = [torch.tensor(output[0]["score"] - script_args.reward_baseline) for output in pipe_outputs]

        # Thực hiện bước PPO
        stats = ppo_trainer.step(question_tensors, response_tensors, rewards)
        ppo_trainer.log_stats(stats, batch, rewards)

        # Lưu checkpoint định kỳ
        if script_args.save_freq and epoch and epoch % script_args.save_freq == 0:
            save_dir = os.path.join(script_args.output_dir, f"step_{epoch}")
            ppo_trainer.save_pretrained(save_dir)
            print(f"Saved checkpoint at: {save_dir}")

    # Lưu checkpoint cuối cùng
    final_save_dir = os.path.join(script_args.output_dir, "step_saved")
    ppo_trainer.save_pretrained(final_save_dir)
    print(f"Final checkpoint saved at: {final_save_dir}")

# Chạy hàm
tuning_lm_with_rl(args)

# # Gộp adapter LoRA
from predict_module.merge_peft_adapter import merge_peft_adapter
merge_peft_adapter(
    model_name=os.path.join(args.output_dir, "step_saved"),
    output_name="./saved_models/sep_model"
)

reward_model_name: ./saved_models/reward_model_deepseek-r1-distill-qwen-adapter-merged
dataset_name: ./datasets/
model_name: ./saved_models/lora-DeepSeek-R1-Distill-Qwen-adapter-merged
train_dataset size: 3


/root/.venv/lib/python3.10/site-packages/trl/trainer/ppo_config.py:207: FutureWarning: `PPOConfig` is deprecated and will be removed in the future. Please use `PPOv2Config` with `PPOv2Trainer` instead.
  warnings.warn(


Tokenizer loaded: LlamaTokenizerFast


Map:   0%|          | 0/64 [00:00<?, ? examples/s]

Dataset created with 64 samples
Finetune model: ./saved_models/lora-DeepSeek-R1-Distill-Qwen-adapter-merged <class 'trl.models.modeling_value_head.AutoModelForCausalLMWithValueHead'>
Dataset({
    features: ['query', 'input_ids'],
    num_rows: 64
})


...

Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at ./saved_models/reward_model_deepseek-r1-distill-qwen-adapter-merged and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0
0it [00:00, ?it/s]You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/root/.venv/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
10it [05:20, 32.65s/it]You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency p

Final checkpoint saved at: ./saved_models/tuning_deepseek_r1_distill_qwen_checkpoints/step_saved
Cấu hình LoRA: LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='./saved_models/lora-DeepSeek-R1-Distill-Qwen-adapter-merged', revision=None, inference_mode=True, r=8, target_modules={'q_proj', 'v_proj', 'o_proj', 'k_proj'}, exclude_modules=None, lora_alpha=16, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)


Lưu mô hình gộp tại: ./saved_models/sep_model


/root/.venv/lib/python3.10/site-packages/transformers/modeling_utils.py:3391: UserWarning: Attempting to save a model with offloaded modules. Ensure that unallocated cpu memory exceeds the `shard_size` (5GB default)
  warnings.warn(


Saving checkpoint shards:   0%|          | 0/1 [00:00<?, ?it/s]

##### SUMMARIZE Tập Test

In [ ]:
# from explain_module.agents import PredictReflectAgent
# from data_load.dataloader import DataLoader

# dataloader = DataLoader(args)

# print("Loading Test Agents...")
# data_test = dataloader.load(flag="test")

# # Lưu DataFrame vào tệp CSV
# path = args.llm_summarize+"_data_test.csv"
# data_test.to_csv(path, index=False)  # index=False để không lưu chỉ số dòng
# print(f"DataFrame đã được lưu vào '{path}'")


##### Thêm Technical Indicator

In [ ]:

# import os
# import pandas as pd
# import pandas_ta as ta  # Sử dụng pandas_ta vì dễ cài đặt và tích hợp với pandas
# import numpy as np

# def format_technical_indicators(row):
#     """
#     Chuyển đổi một dòng của DataFrame thành văn bản, bỏ qua giá trị NaN.

#     Args:
#         row (pd.Series): Một dòng của DataFrame chứa các chỉ số kỹ thuật.

#     Returns:
#         str: Văn bản chứa chỉ số kỹ thuật, mỗi dòng cách nhau bởi '\n'.
#     """
#     lines = []
    
#     for col, value in row.items():
#         if pd.notna(value):  # Bỏ qua nếu giá trị là NaN
#             lines.append(f"{col}: {value:,}")  # Định dạng số với dấu phẩy
    
#     return "\n".join(lines)  # Kết hợp các dòng thành văn bản


# # Đọc file CSV vào DataFrame
# data = pd.read_csv("data_test.csv")


# def add_technical_indicator(data):
#     # Lấy danh sách các giá trị duy nhất của cột "ticker"
#     unique_tickers = data["ticker"].unique()
#     unique_tickers

#     technical_indicator_dir = "data/sample_price/technical_indicator/"
#     # DataFrame để lưu kết quả cuối cùng
#     final_df = pd.DataFrame()


#     for file_name in unique_tickers:

#         technical_indicator_path = os.path.join(technical_indicator_dir, file_name + ".csv")

#         df_technical_indicator = pd.read_csv(technical_indicator_path)

#         # Loại bỏ cột 'Date' để chỉ giữ các chỉ báo kỹ thuật
#         df_technical_indicator["technical_indicator"] = df_technical_indicator.drop(columns=["Date"]).apply(format_technical_indicators, axis=1)

#         # Thêm cột "ticker" với giá trị file_name
#         df_technical_indicator["ticker"] = file_name

#         # Hiển thị kết quả  
#         df_technical_indicator = df_technical_indicator[["ticker", "Date", "technical_indicator"]].rename(columns={"Date": "date"})
#         # Merge theo 2 cột "ticker" và "Date"
#         merged_df = pd.merge(data, df_technical_indicator, on=["ticker", "date"], how="inner")

#         # Cộng dồn kết quả
#         final_df = pd.concat([final_df, merged_df], ignore_index=True)

#     return final_df

    

# data = add_technical_indicator(data)
# data

# data.to_csv("data_test.csv", index=False)


##### CHẠY TẬP TEST
*** chú ý: Kích thước tập Test chưa được chạy toàn bộ (data = data[:2])

In [5]:
import pandas as pd
from explain_module.agents import PredictReflectAgent
from explain_module.util import summarize_trial, remove_reflections, save_results#, save_agents

# Đọc tệp CSV
data = pd.read_csv(args.llm_summarize+"_data_test.csv")
data = data[:2]
data


# agent_cls = PredictReflectAgent
# test_agents = [agent_cls(row['ticker'], row['summary'], row['target']) for _, row in data.iterrows()]
# print("Loaded Test Agents.")
# test_agents

,ticker,summary,target,date,technical_indicator
0,AMZN,2021-12-05\nAmazon (AMZN) is making its own co...,Negative,2021-12-10,SMA_5: 174.01\nEMA_5: 173.63\nMACD: 0.05\nMACD...
1,AMZN,2021-12-29\n- Amazon's stock symbol is $AMZN.\...,Positive,2022-01-03,SMA_5: 169.13\nEMA_5: 169.2\nMACD: -1.23\nMACD...


In [ ]:
import pandas as pd
from explain_module.agents import PredictReflectAgent
from explain_module.util import summarize_trial, remove_reflections, save_results#, save_agents
from trl import AutoModelForCausalLMWithValueHead
from transformers import GenerationConfig, pipeline


def test(args):

    # Đọc tệp CSV
    data = pd.read_csv(args.llm_summarize+"_data_test.csv")
    data = data[:2] # Giảm kích thước tập Test để chạy nhanh hơn


    #==========================================================================
    # print("Loading Test Agents...")
    # data = self.dataloader.load(flag="test")
    # print("Test data loaded with columns:", data.columns.tolist())
    # print("Number of test samples:", len(data))
    #==========================================================================

    agent_cls = PredictReflectAgent
    test_agents = [agent_cls(row['ticker'], row['summary'], row['target'], row['technical_indicator'], '') for _, row in data.iterrows()]
    print("Loaded Test Agents:", len(test_agents))


    tokenizer_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLMWithValueHead.from_pretrained(
        "./saved_models/sep_model",
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    )

    reward_model = pipeline(
        "sentiment-analysis",
        model=args.reward_model_name,
        device_map="auto",
        model_kwargs={"torch_dtype": torch.bfloat16},
        tokenizer=tokenizer
    )

    for agent in test_agents:
        agent.run_n_shots(
            model=model,
            tokenizer=tokenizer,
            reward_model=reward_model,
            num_shots=args.num_shots
        )

    correct, incorrect = summarize_trial(test_agents)
    print(f'Finished evaluation, Correct: {len(correct)}, Incorrect: {len(incorrect)}')

    save_results(test_agents, args.save_dir)
    print(f"kết quả được lưu vào fie {args.save_dir}")

test(args)

##### Đọc Kết Quả và tính Metric

In [4]:
import pandas as pd
df_result = pd.read_csv("results/results.csv")
df_result

# Tách cột 'Response' thành hai cột mới
df_result[['Trend', 'Explanation']] = df_result['Response'].str.split(r'\nExplanation:\s*', expand=True)
df_result # Đặt lại thứ tự cột
df_result = df_result[['Prompt', 'Response', 'Trend', 'Explanation', 'Target']]
df_result



,Prompt,Response,Trend,Explanation,Target
0,Given a list of facts and a set of technical i...,Positive\nExplanation: Amazon's strong fourth...,Positive,"Amazon's strong fourth-quarter performance, dr...",Negative
1,Given a list of facts and a set of technical i...,Positive\nExplanation: Amazon's strong Q3 perf...,Positive,"Amazon's strong Q3 performance, strong cash re...",Positive


Tính Accuracy và MCC

In [5]:
from sklearn.metrics import accuracy_score, matthews_corrcoef

# Giả sử df là DataFrame của bạn
y_pred = df_result["Trend"]
y_true = df_result["Target"]

# Tính accuracy
acc = accuracy_score(y_true, y_pred)

# Tính MCC
mcc = matthews_corrcoef(y_true, y_pred)

print(f"Accuracy: {acc:.4f}")
print(f"MCC: {mcc:.4f}")


Accuracy: 0.5000
MCC: 0.5000


In [11]:
# sep-main-update\result_chatgpt\TEST_OpenAILLM_top1_stock_technical_indicator_test_1.jsonresults.csv

import pandas as pd
import os

# Thư mục chứa các file kết quả CSV
folder = "./result_chatgpt"

# Tạo danh sách file cần ghép
csv_files = [
    os.path.join(folder, f"TEST_OpenAILLM_top1_stock_technical_indicator_test_{i}.jsonresults.csv")
    for i in range(1, 12)  # nếu bạn có 11 phần
]

# Đọc và ghép tất cả các file CSV
df_all = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)

# Hiển thị vài dòng đầu
df_result = df_all
# df_result

# Tách cột 'Response' thành hai cột mới
# Tách trend (Positive/Negative) ở dòng đầu
df_result['Trend'] = df_result['Response'].str.split(r'\nExplanation:\s*').str[0].str.strip()

# Tách Explanation từ '\nExplanation:' đến trước '\n' kế tiếp (nếu có)
df_result['Explanation'] = df_result['Response'].str.extract(r'\nExplanation:\s*(.*?)(?:\n|$)', expand=False)
df_result # Đặt lại thứ tự cột
df_result = df_result[['Prompt', 'Response', 'Trend', 'Explanation', 'Target']]
df_result

,Prompt,Response,Trend,Explanation,Target
0,Given a list of facts and a set of technical i...,Positive\n\nExplanation: Despite mixed market ...,Positive,Despite mixed market performance and ongoing d...,Negative
1,Given a list of facts and a set of technical i...,Negative\n\nExplanation: Despite some technica...,Negative,Despite some technical indicators suggesting a...,Positive
2,Given a list of facts and a set of technical i...,Positive\n\nExplanation: The technical indicat...,Positive,The technical indicators show a mixed signal w...,Negative
3,Given a list of facts and a set of technical i...,Positive\n\nExplanation: Despite some negative...,Positive,Despite some negative sentiment in the tech se...,Positive
4,Given a list of facts and a set of technical i...,Neutral\n\nExplanation: The technical indicato...,Neutral,The technical indicators suggest that the stoc...,Negative
...,...,...,...,...,...
163,Given a list of facts and a set of technical i...,Negative\n\nExplanation: Despite relatively ne...,Negative,Despite relatively neutral technical indicator...,Negative
164,Given a list of facts and a set of technical i...,Negative\n\nExplanation: Despite having a rela...,Negative,Despite having a relatively stable SMA and EMA...,Negative
165,Given a list of facts and a set of technical i...,Negative\n\nExplanation: Despite having a slig...,Negative,"Despite having a slightly higher SMA and EMA, ...",Positive
166,Given a list of facts and a set of technical i...,Negative\n\nExplanation: Despite some positive...,Negative,Despite some positive factors like an increase...,Negative


In [12]:
from sklearn.metrics import accuracy_score, matthews_corrcoef

# Giả sử df là DataFrame của bạn
y_pred = df_result["Trend"]
y_true = df_result["Target"]

# Tính accuracy
acc = accuracy_score(y_true, y_pred)

# Tính MCC
mcc = matthews_corrcoef(y_true, y_pred)

print(f"Accuracy: {acc:.4f}")
print(f"MCC: {mcc:.4f}")


Accuracy: 0.5238
MCC: 0.0801
